This notebook is aiming at prepare training data for third classifier for admit group of multi-step strategies. It also contains the part to kick off training jobs after data is prepared. (Training job will use "auto-split" mode to reserve validation data). Also, this training set includes the keyword records prepared in other notebook.

Prerequisite:

1) csv files containing extracted text files

2) A csv file prepared in Step2 notebook, containing label information corresponding to text files

3) Keyword records prepared in step2.5 notebook

Output:

1) Prepared training file for admit group classifier

2) Kick off binary classification training job

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import boto3

In [2]:
# Load prior prepared extracted text files
textract_df = pd.read_csv("Textract_output_train04.csv")
textract_df.rename(columns={'file_name' : 'filename'}, inplace=True)
textract_df

filename  page  \
0                 1003700000000530903_11159719.pdf     1   
1                 1003700000000530903_11159719.pdf     2   
2                 1003700000000530903_11159719.pdf     3   
3                 1003700000000530903_11159719.pdf     4   
4                 1003700000000530903_11159719.pdf     5   
...                                            ...   ...   
840163  OCR_33049_1003700000000697103_11103804.pdf    43   
840164  OCR_33049_1003700000000697103_11103804.pdf    44   
840165  OCR_33049_1003700000000697103_11103804.pdf    45   
840166  OCR_33049_1003700000000697103_11103804.pdf    46   
840167  OCR_33049_1003700000000697103_11103804.pdf    47   

                                                  content  
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...  
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...  
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...  
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...  
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...  
...                                                   ...  
840163  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  
840164  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840165  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840166  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840167  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  

[840168 rows x 3 columns]

In [3]:
# Change filename to same format 
textract_df['filename'] = textract_df['filename'].str.split('_').str[-2] + '_' + textract_df['filename'].str.split('_').str[-1]

In [4]:
# Load prior prepared reformat label file
label_df = pd.read_csv("batch3_label_revised.csv")
label_df

page             class                          filename  filename_cut
0         29   admission_order  1003700000000633142_11068501.pdf        633142
1         43  history_physical  1003700000000633142_11068501.pdf        633142
2         44  history_physical  1003700000000633142_11068501.pdf        633142
3         45  history_physical  1003700000000633142_11068501.pdf        633142
4         46  history_physical  1003700000000633142_11068501.pdf        633142
...      ...               ...                               ...           ...
357778    81  history_physical  1003700000000675009_11251999.pdf        675009
357779    82  history_physical  1003700000000675009_11251999.pdf        675009
357780    83  history_physical  1003700000000675009_11251999.pdf        675009
357781    84  history_physical  1003700000000675009_11251999.pdf        675009
357782    85  history_physical  1003700000000675009_11251999.pdf        675009

[357783 rows x 4 columns]

In [5]:
# In the label file, only keeping the labels for AT/AR/AO classes
new_label_df = pd.DataFrame()
at_df = label_df[label_df['class']=='admit_type']
ar_df = label_df[label_df['class']=='admission_reason']
ao_df = label_df[label_df['class']=='admission_order']
new_label_df = pd.concat([new_label_df, at_df, ar_df, ao_df], axis=0, ignore_index=True)

In [7]:
# Check the distribution and see if it satisfies the 10:1 ratio
new_label_df['class'].value_counts()

admission_order     9082
admission_reason    6782
admit_type          6162
Name: class, dtype: int64

In [8]:
# For consistency, rename the new_labe_df back to label_df
label_df = new_label_df

In [10]:
# Drop duplicate records
label_df.drop_duplicates(inplace=True)
label_df

page            class                          filename  filename_cut
0         2       admit_type  1003700000000642572_11072823.pdf        642572
1         3       admit_type  1003700000000642572_11072823.pdf        642572
2         2       admit_type  1003700000000680010_11073524.pdf        680010
3         2       admit_type  1003700000000633142_11068501.pdf        633142
4        29       admit_type  1003700000000633142_11068501.pdf        633142
...     ...              ...                               ...           ...
22021    19  admission_order  1003700000000737088_11248398.pdf        737088
22022    20  admission_order  1003700000000737088_11248398.pdf        737088
22023    29  admission_order  1003700000000737088_11248398.pdf        737088
22024    30  admission_order  1003700000000737088_11248398.pdf        737088
22025    32  admission_order  1003700000000737088_11248398.pdf        737088

[22017 rows x 4 columns]

In [11]:
# Check common file
common_files = set(textract_df.filename).intersection(label_df.filename)
common_files_train = common_files
len(common_files_train)

2368

In [12]:
# filter out uncommon file in text dataframe
textract_df_train = textract_df[textract_df.filename.isin(common_files_train)]
textract_df_train

filename  page  \
0       1003700000000530903_11159719.pdf     1   
1       1003700000000530903_11159719.pdf     2   
2       1003700000000530903_11159719.pdf     3   
3       1003700000000530903_11159719.pdf     4   
4       1003700000000530903_11159719.pdf     5   
...                                  ...   ...   
840163  1003700000000697103_11103804.pdf    43   
840164  1003700000000697103_11103804.pdf    44   
840165  1003700000000697103_11103804.pdf    45   
840166  1003700000000697103_11103804.pdf    46   
840167  1003700000000697103_11103804.pdf    47   

                                                  content  
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...  
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...  
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...  
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...  
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...  
...                                                   ...  
840163  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  
840164  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840165  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840166  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840167  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  

[833497 rows x 3 columns]

In [13]:
# filter out uncommon file in label dataframe
label_df_train = label_df[label_df.filename.isin(common_files_train)]
label_df_train

page            class                          filename  filename_cut
0         2       admit_type  1003700000000642572_11072823.pdf        642572
1         3       admit_type  1003700000000642572_11072823.pdf        642572
2         2       admit_type  1003700000000680010_11073524.pdf        680010
3         2       admit_type  1003700000000633142_11068501.pdf        633142
4        29       admit_type  1003700000000633142_11068501.pdf        633142
...     ...              ...                               ...           ...
22021    19  admission_order  1003700000000737088_11248398.pdf        737088
22022    20  admission_order  1003700000000737088_11248398.pdf        737088
22023    29  admission_order  1003700000000737088_11248398.pdf        737088
22024    30  admission_order  1003700000000737088_11248398.pdf        737088
22025    32  admission_order  1003700000000737088_11248398.pdf        737088

[21981 rows x 4 columns]

In [14]:
# merge text files with labels
merged_file_train = pd.merge(textract_df_train, label_df_train, how = "left", on = ["filename", "page"])
merged_file_train

filename  page  \
0       1003700000000530903_11159719.pdf     1   
1       1003700000000530903_11159719.pdf     2   
2       1003700000000530903_11159719.pdf     3   
3       1003700000000530903_11159719.pdf     4   
4       1003700000000530903_11159719.pdf     5   
...                                  ...   ...   
836045  1003700000000697103_11103804.pdf    43   
836046  1003700000000697103_11103804.pdf    44   
836047  1003700000000697103_11103804.pdf    45   
836048  1003700000000697103_11103804.pdf    46   
836049  1003700000000697103_11103804.pdf    47   

                                                  content class  filename_cut  
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...   NaN           NaN  
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...   NaN           NaN  
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...   NaN           NaN  
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...   NaN           NaN  
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...   NaN           NaN  
...                                                   ...   ...           ...  
836045  Patient: GABRY, KAREN11800 E. Twelve Mile Road...   NaN           NaN  
836046  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...   NaN           NaN  
836047  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...   NaN           NaN  
836048  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...   NaN           NaN  
836049  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...   NaN           NaN  

[836050 rows x 5 columns]

In [15]:
# Drop unlabelled records as we already removed "others" class in first step
merged_file_train.dropna(subset=['class'], inplace=True)
merged_file_train

filename  page  \
15      1003700000000530903_11159719.pdf    16   
30      1003700000000530903_11159719.pdf    31   
35      1003700000000530903_11159719.pdf    36   
45      1003700000000530903_11159719.pdf    46   
77      1003700000000530903_11159719.pdf    78   
...                                  ...   ...   
836005  1003700000000697103_11103804.pdf     5   
836007  1003700000000697103_11103804.pdf     7   
836008  1003700000000697103_11103804.pdf     7   
836009  1003700000000697103_11103804.pdf     7   
836014  1003700000000697103_11103804.pdf    12   

                                                  content             class  \
15      University of Kentucky HospitalOptum Medical N...   admission_order   
30      Documents Review ReportxPatient: BAKER, EDWINN...   admission_order   
35      Documents Review ReportPatient: BAKER, EDWINNA...  admission_reason   
45      Documents Review ReportPatient:BAKER, EDWINNA ...   admission_order   
77      Documents Review ReportPatient: BAKER, EDWINNA...   admission_order   
...                                                   ...               ...   
836005  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  admission_reason   
836007  Patient: GABRY, KAREN11800 E. Twelve Mile Road...        admit_type   
836008  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  admission_reason   
836009  Patient: GABRY, KAREN11800 E. Twelve Mile Road...   admission_order   
836014  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  admission_reason   

        filename_cut  
15          530903.0  
30          530903.0  
35          530903.0  
45          530903.0  
77          530903.0  
...              ...  
836005      697103.0  
836007      697103.0  
836008      697103.0  
836009      697103.0  
836014      697103.0  

[22059 rows x 5 columns]

In [16]:
# Prepare data ready for "multi-label" mode
merged_file_train = merged_file_train[['content', 'class']].drop_duplicates().groupby('content')['class'].apply('|'.join).reset_index()

In [17]:
# Add most recent keyword records (prepared in other notebooks)
keyword_file = "keywords_to_training_v2.csv"
keyword_df = pd.read_csv(keyword_file)
keyword_df

class                      content
0                admit_type  Admission status: Inpatient
1                admit_type    admission for observation
2                admit_type             admission orders
3                admit_type   admission status inpatient
4                admit_type       admission to inpatient
..                      ...                          ...
700  nutritional_assessment         Nutritional Problems
701  nutritional_assessment    nutritional re-assessment
702  nutritional_assessment        Reason for Assessment
703  nutritional_assessment     Service  Type: Nutrition
704  nutritional_assessment           service:nutrtition

[705 rows x 2 columns]

In [18]:
# In the keyword file, only keeping the records of AT/AR/AO classes
new_keyword_df = pd.DataFrame()
at_df = keyword_df[keyword_df['class']=='admit_type']
ar_df = keyword_df[keyword_df['class']=='admission_reason']
ao_df = keyword_df[keyword_df['class']=='admission_order']
new_keyword_df = pd.concat([new_keyword_df, at_df, ar_df, ao_df], axis=0, ignore_index=True)

In [20]:
# Concatenate training records with keyword records
merged_file_train = pd.concat([merged_file_train, new_keyword_df], axis=0, ignore_index = True)
merged_file_train

content  \
0      !! UNABLE TO FIND 4 tablets every morning. Med...   
1      " John J Worch Jr. is a 76 y.o. male with a hi...   
2      "- No tobacco use in the past 30 days.Conditio...   
3      # PAD- avoid contrast if possible# hypertensio...   
4      # START~1356072~51260774~HCRDPN~##ENDHMC VRF D...   
...                                                  ...   
19500                                     order to admit   
19501                                     orders written   
19502                            patient class inpatient   
19503                              Patient status change   
19504                                   Status:Inpatient   

                                  class  
0      admission_reason|admission_order  
1                            admit_type  
2      admission_reason|admission_order  
3                       admission_order  
4                      admission_reason  
...                                 ...  
19500                   admission_order  
19501                   admission_order  
19502                   admission_order  
19503                   admission_order  
19504                   admission_order  

[19505 rows x 2 columns]

In [21]:
# Reorder the training dataframe to satisfy Comprehend's requirement
merged_file_train = merged_file_train[['class', 'content']]

In [22]:
# Save training data locally
merged_file_train.to_csv('multistep-3-admitgroup-training-data-autosplit-keyword.csv', index=False, header=False)

In [23]:
# Specify region and client
REGION = 'us-east-1'
s3 = boto3.resource('s3')
comprehend = boto3.client('comprehend', region_name = REGION)

In [24]:
# upload training file to S3 (we're uploading to a folder that training pipeline is not monitoring, so we can manually kickoff jobs)
s3.Bucket('scio-saas-datastores').upload_file('multistep-3-admitgroup-training-data-autosplit-keyword.csv', 'SaaS_workspace/aws_comprehend/inputs_manual/multistep-3-admitgroup-training-data-autosplit-keyword.csv')

In [25]:
# Specify training parameters (classifier name has to be UNIQUE)
name = 'multistep-3a-custom-classifier-admitgroup-yy-autosplit-keyword'
language_code = 'en'
training_bucket = 'scio-saas-datastores'
training_key = 'SaaS_workspace/aws_comprehend/inputs_manual/multistep-3-admitgroup-training-data-autosplit-keyword.csv'
data_access_role_arn = 'arn:aws:iam::097079433570:role/service-role/AmazonComprehendServiceRole-aws_scio_S3Datastore_servicewrite'

In [26]:
# Kick off training job
response = comprehend.create_document_classifier(
        DocumentClassifierName = name,
        LanguageCode = language_code,
        InputDataConfig = {
            "DataFormat": "COMPREHEND_CSV",
            "S3Uri":f"s3://{training_bucket}/{training_key}"
        },

        OutputDataConfig = {
            "S3Uri":"s3://scio-saas-datastores/SaaS_workspace/aws_comprehend/output/batch3-multistep",
            "KmsKeyId": "arn:aws:kms:us-east-1:097079433570:key/1c6d057c-0be3-40a6-b23c-5870a92b901f"
        },

        DataAccessRoleArn = data_access_role_arn,
        VolumeKmsKeyId = "arn:aws:kms:us-east-1:097079433570:key/1c6d057c-0be3-40a6-b23c-5870a92b901f",
        Mode = "MULTI_LABEL",

)